These are directions that can be followed in order to add genes to the standard PICRUSt2 database using a Hidden Markov Model (HMM) for that gene (or genes)
The PICRUSt2 genomes will need to be downloaded, decompressed and saved somewhere locally.
They can be downloaded from: https://doi.org/10.6084/m9.figshare.12233192
And they can be decompressed by running 'tar -xf path_to_file/JGI_PICRUSt_genomes.tar.bz2' from the terminal (where path_to_file is replaced with the actual path to the file or removed if running from inside the same folder)

You will also need to ensure that several packages are installed:
- Conda: https://docs.conda.io/projects/conda/en/latest/commands/install.html
- HMM, which can be done by running the following in the terminal: 'conda install -c biocore hmmer in terminal'
- Biopython, which can be done by running the following in the terminal 'conda install biopython'

You will also need the 'ko.txt.gz' file that should be in the same folder as this script. If not, you can download this from the PICRUSt2 github page here: https://github.com/picrust/picrust2/blob/master/picrust2/default_files/prokaryotic/ko.txt.gz

You should also replace the HMMs in the '/hmms/' folder with HMMs for the genes that you are interested in. 
The HMMs that are currently shown were made from the .fasta files in the 'hmms_to_make' folder. To make these, you can follow these steps:
1. Search for the top hits of the gene of interest in uniprot (https://www.uniprot.org/)
2. Click on the genes that you want to include and follow the link for the genomic DNA translation
3. Combine all of the DNA sequences into one .fasta file (you can do this using a text editing software)
4. Get a stockholm alignment of the .fasta file. We used https://www.ebi.ac.uk/Tools/msa/clustalo/ (select 'DNA' and the 'STOCKHOLM' alignment option)
5. Download this alignment and run 'hmmbuild PETase_DNA.hmm PETase_DNA.sto' in terminal
6. Move the .hmm file to the 'hmms/' folder

1. Import the packages that we will need

In [54]:
from Bio import SeqIO
import os
import csv
import pandas as pd
from optparse import OptionParser
from Bio.Alphabet import generic_dna, generic_protein

2. Now give the paths to the files that we are using, changing these if necessary

In [55]:
picrust_seqs = 'JGI_PICRUSt_genomes.fasta'
hmms = os.listdir(os.getcwd()+'/hmms/')
ko = 'ko.txt'

3. Open these files and set up the directories that we will save things to

In [56]:
os.system('gunzip '+ko+'.gz')
try: os.mkdir('hmms_out')
except: didnt_make = True
ko_data = pd.read_csv(ko, header=0, index_col=0, sep='\t')

4. Perform the HMM searches of the PICRUSt2 sequences using your HMMs (this will take a while to run)

In [32]:
for hmm in hmms:
    os.system('nhmmer hmms/'+hmm+' '+picrust_seqs+' > hmms_out/'+hmm[:-4]+'.out')

You can open any of the files in the hmms_out folder if you want to check whether you have any hits that are above the inclusion threshold (and whether this fits what you would have expected)

5. Now take the information from these HMMs and add this to the PICRUSt2 KEGG ortholog information that we already have (this is a bit tedious as the HMM.out files don't use tabs between columns or anything that we could use to separate them, so we just have to read them in as text files and look at each character...)

In [57]:
hmms_out = os.listdir(os.getcwd()+'/hmms_out')
main_dir = os.getcwd()
genomes = list(ko_data.index.values)
genomes = [str(genomes[i]).replace('-cluster', '') for i in range(len(genomes))]
for hmm in hmms_out:
    included_genomes = []
    with open(main_dir+'/hmms_out/'+hmm, 'rU') as f:
        contents = f.read()
    row, rows = '', []
    for a in range(len(contents)-1):
        if contents[a:a+1] == '\n':
            if row == '  ------ inclusion threshold ------':
                break
            rows.append(row)
            row = ''
        else:
            row += contents[a]
    after_start, other_count = False, 0
    for r in range(len(rows)):
        if after_start:
            block = 0
            this_genome = ''
            for b in range(1, len(rows[r])):
                if rows[r][b-1] == ' ' and rows[r][b] != ' ':
                    block += 1
                if block == 4 and rows[r][b] != ' ':
                    this_genome += rows[r][b]
            if this_genome != '':
                included_genomes.append(this_genome)
        count = 0
        for a in range(len(rows[r])):
            if rows[r][a] == '-':
                count += 1
            if count > 40:
                after_start = True
                continue
    for a in range(len(included_genomes)):
        if included_genomes[a][-11:] == 'Description':
            included_genomes[a] = included_genomes[a][:-11]
    this_col = []
    for g in genomes:
        c1 = included_genomes.count(g)
        c2 = included_genomes.count(g[:-8])
        this_col.append(c1+c2)
    ko_data[hmm[:-4]] = this_col
ko_data.to_csv('ko_all.txt', sep='\t')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 'U' mode is deprecated
  import sys


You can now check the ko_all.txt file, but there should be new columns titled with your HMM names and counts of how many times these genes are in each of your genomes in the rows. If you want to use these with the rest of the Plastisphere metaanalysis then you should replace the 'ko_all.txt' file in the picrust folders in both of the folders inside the 'all_output_and_recreate' folder (downloaded from https://doi.org/10.6084/m9.figshare.12227303.v3)